In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
import numpy as np
import os
import time

import utils

import networks

import matplotlib.pyplot as plt

import importlib

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


# Data Sampling
Here in our case, the system is European Call Option PDE and the physical information about the system consists of Boundary Value conditions, final Value conditions and the PDE itself.

In [3]:
K = 10
r = 0.035
sigma = 0.2
T = 1
S_range = [0, int(5*K)]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)
M = 100
N = 5000

# Build Neural Network

In [4]:
pinn = networks.FeedforwardNeuralNetwork(2, 50, 1, 8)
pinn.cuda()

FeedforwardNeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1-7): 7 x Linear(in_features=50, out_features=50, bias=True)
  )
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)

In [5]:
n_epochs = 5000
lossFunction = nn.MSELoss()
optimizer = optim.Adam(pinn.parameters(), lr=0.00003)

# physical loss samples
samples = {"pde": 5000, "bc":500, "fc":500}

# sample data generated by finite difference method
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor = utils.fdm_data(S_range[-1], T, M, N, "500000sample.csv", device)

# Modelling

- For each iteration in the training loop, we are sampling data for the three physical conditions of the PDE.
- Then we are calculating the loss three times on the same model, accumulating them into a combined objective function to be minimised for the Neural Network.
- The first loss is the differential equation loss. Here we are trying to minimise the PDE by calculating gradients and forming the PDE itself.
- The remaining losses are calculated for boundary value and initial value conditions for the PDE.
- Mean Squared Error loss function `nn.MSELoss()` is chosen as the criterion to be minimised and 
- Adam optimizer `nn.optim.Adam(lr=3e-5)` with a learning rate of 0.00003 is chosen for performing the weight updates.

In [6]:
loss_hist = []
start_time = time.time()


for epoch in range(n_epochs):
    
    i_st_train, i_v_train, bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData3(K, r, sigma, T, S_range[-1], S_range, t_range, gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    
    # save training data points to tensor and send to device
    i_st_train = torch.from_numpy(i_st_train).float().requires_grad_().to(device)
    i_v_train = torch.from_numpy(i_v_train).float().to(device)
    
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)
    
    
    # PDE Round
    y1_hat = pinn(n_st_train)
    grads = tgrad.grad(y1_hat, n_st_train, grad_outputs=torch.ones(y1_hat.shape).cuda(), retain_graph=True, create_graph=True, only_inputs=True)[0]
    # print(grads)
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    # print(grads2nd)
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*y1_hat)
    
    # BC Round
    y2_hat = pinn(bc_st_train)
    bc_loss = lossFunction(bc_v_train, y2_hat)
    
    # initial condition loss
    ic_hat = pinn(i_st_train)
    ic_loss = lossFunction(i_v_train, ic_hat)
    
    # data Round
    y3_hat = pinn(X_train_tensor)
    data_loss = lossFunction(y_train_tensor, y3_hat)
    
    
    # Backpropagation and Update
    optimizer.zero_grad()
    combined_loss = pde_loss + bc_loss + ic_loss + data_loss
    combined_loss.backward()
    optimizer.step()
    
    
    # print loss
    mse_loss = pde_loss + bc_loss + ic_loss + data_loss
    loss_hist.append(mse_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{n_epochs} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.6f}, fc loss: {ic_loss.item(): 5f}, L2 loss: {mse_loss.item():6f}, minimum loss: {min(loss_hist):.5f}')

end_time = time.time()
print('run time:', end_time - start_time)

0/5000 PDE Loss: 0.00001, BC Loss: 804.588684, fc loss:  408.470215, L2 loss: 1635.338745, minimum loss: 1635.33875
500/5000 PDE Loss: 0.00029, BC Loss: 57.709110, fc loss:  23.134628, L2 loss: 105.608719, minimum loss: 105.60872
1000/5000 PDE Loss: 0.41290, BC Loss: 1.539094, fc loss:  4.968821, L2 loss: 12.554564, minimum loss: 12.55456
1500/5000 PDE Loss: 0.05357, BC Loss: 0.007794, fc loss:  0.014565, L2 loss: 0.101519, minimum loss: 0.10152
2000/5000 PDE Loss: 0.00694, BC Loss: 0.000290, fc loss:  0.002815, L2 loss: 0.012616, minimum loss: 0.01259
2500/5000 PDE Loss: 0.00222, BC Loss: 0.000169, fc loss:  0.002142, L2 loss: 0.006678, minimum loss: 0.00668
3000/5000 PDE Loss: 0.00139, BC Loss: 0.000068, fc loss:  0.001848, L2 loss: 0.005426, minimum loss: 0.00543
3500/5000 PDE Loss: 0.00123, BC Loss: 0.000134, fc loss:  0.001836, L2 loss: 0.005414, minimum loss: 0.00509
4000/5000 PDE Loss: 0.00122, BC Loss: 0.000027, fc loss:  0.001610, L2 loss: 0.005030, minimum loss: 0.00497
4500/